In [1]:
#| default_exp main
import sys,os
from fastcore.xtras import Path
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
%load_ext autoreload
%autoreload 2

# Principal
> Este módulo concentra funções auxiliares específicas que filtram os dados do banco com campos e formatação de interesse para aplicações específicas como o [appAnalise](https://github.com/EricMagalhaesDelgado/appAnalise) por exemplo.

In [2]:
#| export
import json
from datetime import datetime
from pathlib import Path
from typing import Union

import pandas as pd
import pyodbc
from dotenv import find_dotenv, load_dotenv
from fastcore.test import *
from pymongo import MongoClient
from rich import print

from extracao.format import _filter_matlab, _format_matlab, merge_on_frequency

load_dotenv(find_dotenv(), override=True)

True

In [3]:
#|eval: false
folder = Path.cwd().parent / 'extracao' / 'datasources' / 'arquivos' / 'saida'
folder.ls()

(#28) [Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/aero.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/aero_discarded.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/anatel.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/AnatelDB.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/anatel_discarded.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/arquitetura.md'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/base.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/canalização_smp.xlsx'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/s

In [4]:
#|eval: false
#|code_fold: true

def read_df(name: str)->pd.DataFrame:
    return pd.read_parquet(folder / f'{name}.parquet.gzip', dtype_backend='pyarrow')# .astype("category")

In [5]:
radcom = read_df('radcom')
radcom

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,Classe,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade,Log
0,87.5,ASSOCIACAO COMUNITARIA VOZ DA LIBERDADE DE TUR...,50415095220,231,1008210959,-2.228611111111,-45.306666666666665,2112456,Turilândia,MA,P-A,-1,256,-1,RADCOM,SRD,1,
1,87.5,ASSOCIACAO COMUNITARIA DO MORAD.DE ALVORADA DE...,50409064718,231,699491851,-18.734166666666667,-43.36472222222217,3102407,Alvorada de Minas,MG,3,-1,256,-1,RADCOM,SRD,1,
2,87.5,ASSOCIAÇÃO COMUNITARIA DA JUVENTUDE DE CONGONH...,50405625782,231,699359830,-18.812777777777665,-43.673611111111,3118106,Congonhas do Norte,MG,3,-1,256,-1,RADCOM,SRD,1,
3,87.5,ASSOCIACAO COMUNITARIA FOLHETA,50404381251,231,690859562,-18.95,-43.266666666666666,3122603,Dom Joaquim,MG,3,-1,256,-1,RADCOM,SRD,1,
4,87.5,ASSOCIAÇÃO DE RÁDIO COMUNITÁRIA DE CASTANHEIRA...,50411566547,231,1008401606,-11.137222222222167,-58.61333333333334,5102850,Castanheira,MT,P-K,-1,256,-1,RADCOM,SRD,1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999,107.9,ASSOCIAÇÃO CULT COMUNIT MORUMBI,50407431578,231,697707695,-23.267777777777667,-45.8975,3549904,São José dos Campos,SP,3-B,-1,256,-1,RADCOM,SRD,1,
5000,107.9,ASSOCIAÇÃO DE MORADORES DO JARDIM CRISTINA OUR...,50406232911,231,692270272,-23.5530555555555,-45.8705555555555,3549904,São José dos Campos,SP,3,-1,256,-1,RADCOM,SRD,1,
5001,107.9,ASSOCIAÇÃO DE MORADORES DO JARDIM CRISTINA OUR...,50434484237,231,692270272,-23.5530555555555,-45.8705555555555,3549904,São José dos Campos,SP,3,-1,256,-1,RADCOM,SRD,1,
5002,107.9,ASSOCIACAO COMUNITARIA CULTURAL DE MUSICA E CI...,50406778205,231,693049723,-23.191944444444335,-45.87527777777767,3549904,São José dos Campos,SP,3,-1,256,-1,RADCOM,SRD,1,


In [6]:
stel = read_df('stel')
stel

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,Classe,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade,Log
0,156.75,PETROLEO BRASILEIRO S A PETROBRAS,50411168908,604,1012625432,-24.78901388888883,-42.509736111111,3548500,Santos,SP,ML,G3E,16.0,2033-12-03,L,STEL,1,
1,950.375,"ASSOCIACAO BENEFICENTE, CULTURAL E DE RADIO DI...",50413552748,251,1006805750,-28.354444444444333,-49.29027777777767,4211702,Orleans,SC,TX,F3E,180.0,2022-11-05,L,STEL,1,
2,156.675,PETROLEO BRASILEIRO S A PETROBRAS,50411168908,604,1004309802,-22.432958333333332,-40.480488888888836,3302403,Macaé,RJ,FC,F3E,16.0,2033-12-03,L,STEL,1,
3,156.375,PETROLEO BRASILEIRO S A PETROBRAS,50411168908,604,1012639336,-22.432958333333332,-40.480488888888836,3302403,Macaé,RJ,ML,G3E,11.0,2033-12-03,L,STEL,1,
4,156.8,PETROLEO BRASILEIRO S A PETROBRAS,50411168908,604,1013254799,-2.9618055555555,-38.976,2310209,Paracuru,CE,FC,F3E,16.0,2033-12-03,L,STEL,1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36795,455.962,BAURU RADIO CLUBE S.A.,02031959220,252,3570282,-22.320833333333333,-49.076388888888836,3506003,Bauru,SP,FX,F3E,16.0,1999-06-13,L,STEL,1,
36796,4.369,CLARO S.A.,01020422092,064,895156,-22.965,-43.67333333333333,3304557,Rio de Janeiro,RJ,<NA>,<NA>,<NA>,2031-09-16,L,STEL,1,
36797,156.8,HDG SERVICOS AMBIENTAIS LTDA,50405428871,604,1008115638,-12.690416666666666,-38.63964166666667,2929206,São Francisco do Conde,BA,ML,F3E,11.0,2038-09-12,L,STEL,1,
36798,7747.7,RIC - REDE INTEGRADA DE COMUNICACOES S.A.,50402353137,046,696184788,-29.166388888888832,-51.502777777777666,4302105,Bento Gonçalves,RS,FX,D1D,29700.0,2035-06-06,L,STEL,1,


In [7]:
srd = read_df('srd')
srd

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
0,207.0,REDE DE COMUNICACOES ACREANA LTDA,50442889933,248,[],[],[],1200203,Cruzeiro do Sul,AC,...,<NA>,[],[],[],[],<NA>,0.5,<NA>,<NA>,http://sistemas.anatel.gov.br/se/eApp/reports/...
1,539.0,X-MEDIAGROUP S.A.,50410887137,248,<NA>,<NA>,<NA>,1200336,Mâncio Lima,AC,...,<NA>,<NA>,<NA>,0.0,0.0,<NA>,0.5,[],<NA>,http://sistemas.anatel.gov.br/se/eApp/reports/...
2,79.0,TELEVISAO OESTE BAIANO LTDA,06030116240,248,322647029,-12.1013888888888333,-44.9936111111110000,2903201,Barreiras,BA,...,1000.0,3.20,.00,0.0,44.7,.77,0.5,"{'0': 0, '10': 0.34874388613190943, '20': 0.94...",50.00,http://sistemas.anatel.gov.br/se/eApp/reports/...
3,69.0,TELEVISAO SANTA CRUZ LTDA,06020355110,248,322623553,-14.7794444444443333,-39.2622222222221666,2914802,Itabuna,BA,...,5000.0,2.79,.00,55.0,86.0,0.62,0.25,"{'0': 0.22, '10': 0.0, '20': 0.22, '30': 0.77,...",88.00,http://sistemas.anatel.gov.br/se/eApp/reports/...
4,177.0,TV CABRALIA LTDA,06020354903,248,322623537,-14.7833333333333333,-39.2833333333333333,2914802,Itabuna,BA,...,4700.0,3.22,.00,330,67.41,1.18,0.5,"{'0': 1.01, '10': 1.51, '20': 1.51, '30': 1.01...",75,http://sistemas.anatel.gov.br/se/eApp/reports/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31157,503.0,RF TECNOLOGIA E PARTICIPACOES LTDA,50446946303,801,[],[],[],1303700,Santo Antônio do Içá,AM,...,<NA>,[],[],[],[],[],[],<NA>,[],http://sistemas.anatel.gov.br/se/eApp/reports/...
31158,587.0,RF TECNOLOGIA E PARTICIPACOES LTDA,50446946575,801,[],[],[],5107859,São Félix do Araguaia,MT,...,<NA>,[],[],[],[],[],[],<NA>,[],http://sistemas.anatel.gov.br/se/eApp/reports/...
31159,497.0,RF TECNOLOGIA E PARTICIPACOES LTDA,50446946737,801,[],[],[],1200500,Sena Madureira,AC,...,<NA>,[],[],[],[],[],[],<NA>,[],http://sistemas.anatel.gov.br/se/eApp/reports/...
31160,641.0,RF TECNOLOGIA E PARTICIPACOES LTDA,50446951900,801,[],[],[],5210109,Ipameri,GO,...,<NA>,[],[],[],[],[],[],<NA>,[],http://sistemas.anatel.gov.br/se/eApp/reports/...


In [8]:
telecom = read_df('telecom')
telecom

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,Classe,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade,Log
0,173.075,TRACKER DO BRASIL LTDA,50004466071,011,505805553,-23.4030555555555,-46.63583333333333,3528502,Mairiporã,SP,FB,F2D,13.2,2034-12-23,L,MOSAICO-LICENCIAMENTO,1,"[(""Colunas"", ""Designação_Emissão""]), ..."
1,173.075,TRACKER DO BRASIL LTDA,50004466071,011,684490021,-19.793333333333333,-43.968888888888834,3106200,Belo Horizonte,MG,FB,F2D,13.2,2034-12-23,L,MOSAICO-LICENCIAMENTO,1,"[(""Colunas"", ""Designação_Emissão""]), ..."
2,173.075,TRACKER DO BRASIL LTDA,50004466071,011,684587017,-28.520833333333332,-48.98527777777767,4218707,Tubarão,SC,FB,F2D,13.2,2034-12-23,L,MOSAICO-LICENCIAMENTO,1,"[(""Colunas"", ""Designação_Emissão""]), ..."
3,173.075,TRACKER DO BRASIL LTDA,50004466071,011,684610094,-27.588611111111,-48.533611111111,4205407,Florianópolis,SC,FB,F2D,13.2,2034-12-23,L,MOSAICO-LICENCIAMENTO,1,"[(""Colunas"", ""Designação_Emissão""]), ..."
4,173.075,TRACKER DO BRASIL LTDA,50004466071,011,684649900,-19.868888888888833,-43.193333333333335,3136207,João Monlevade,MG,FB,F2D,13.2,2034-12-23,L,MOSAICO-LICENCIAMENTO,1,"[(""Colunas"", ""Designação_Emissão""]), ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12750,788.0,TELEFONICA BRASIL S.A.,50419058354,011,1014443412,-22.623611,-39.98972,3302403,MACAE,RJ,FB,G7W,10000.0,2029-12-08,L,MOSAICO-LICENCIAMENTO,3,"[(""Colunas"", ""Designação_Emissão""]), ..."
12751,1865.0,TELEFONICA BRASIL S.A.,50419058354,011,1014443447,-21.939722,-39.785278,3205309,Vitória,ES,FB,G7W,10000.0,2029-12-08,L,MOSAICO-LICENCIAMENTO,3,"[(""Colunas"", ""Designação_Emissão""]), ..."
12752,788.0,TELEFONICA BRASIL S.A.,50419058354,011,1014443447,-21.939722,-39.785278,3205309,Vitória,ES,FB,G7W,10000.0,2029-12-08,L,MOSAICO-LICENCIAMENTO,3,"[(""Colunas"", ""Designação_Emissão""]), ..."
12753,1862.5,TELEFONICA BRASIL S.A.,50419058354,011,1014443471,-21.213889,-39.997222,3205309,Vitória,ES,FB,G7W,5000.0,2032-12-22,L,MOSAICO-LICENCIAMENTO,3,"[(""Colunas"", ""Designação_Emissão""]), ..."


In [9]:
smp = read_df('smp')
smp

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,Classe,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade,Log
0,2160.0,TELEFONICA BRASIL S.A.,50409146366,010,3549,-28.5079,-50.9418,4322509,VACARIA,RS,FB,D7W,5000.0,2038-04-30,L,MOSAICO-LICENCIAMENTO,3,"[(""Colunas"", ""Designação_Emissão""]), ..."
1,890.75,TELEFONICA BRASIL S.A.,50409146366,010,3549,-28.5079,-50.9418,4322509,VACARIA,RS,FB,G7W,200.0,2038-04-30,L,MOSAICO-LICENCIAMENTO,13,"[(""Colunas"", ""Designação_Emissão""]), ..."
2,874.5,TELEFONICA BRASIL S.A.,50409146366,010,3549,-28.5079,-50.9418,4322509,VACARIA,RS,FB,G7W,200.0,2038-04-30,L,MOSAICO-LICENCIAMENTO,13,"[(""Colunas"", ""Designação_Emissão""]), ..."
3,2167.5,TELEFONICA BRASIL S.A.,50409146366,010,3549,-28.5079,-50.9418,4322509,VACARIA,RS,FB,G9W,5000.0,2038-04-30,L,MOSAICO-LICENCIAMENTO,3,"[(""Colunas"", ""Designação_Emissão""]), ..."
4,1855.0,TELEFONICA BRASIL S.A.,50409146366,010,3549,-28.5079,-50.9418,4322509,VACARIA,RS,FB,G7W,200.0,2038-04-30,L,MOSAICO-LICENCIAMENTO,9,"[(""Colunas"", ""Designação_Emissão""]), ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22793,1970.0,TELEFONICA BRASIL S.A.,50409146447,010,535800142,-23.1761,-48.1195,3540507,Porangaba,SP,ML,G7W,5000.0,2032-12-22,L,CANALIZACAO-SMP,3,"[(""Colunas"", ""Designação_Emissão""]), ..."
22794,1767.5,TELEFONICA BRASIL S.A.,50409146447,010,535800142,-23.1761,-48.1195,3540507,Porangaba,SP,ML,G7W,5000.0,2032-12-22,L,CANALIZACAO-SMP,3,"[(""Colunas"", ""Designação_Emissão""]), ..."
22795,829.5,TELEFONICA BRASIL S.A.,50409146447,010,535800142,-23.1761,-48.1195,3540507,Porangaba,SP,ML,G7W,5000.0,2032-12-22,L,CANALIZACAO-SMP,3,"[(""Colunas"", ""Designação_Emissão""]), ..."
22796,845.75,TELEFONICA BRASIL S.A.,50409146447,010,689068093,-20.39,-49.4309,3535002,PALESTINA,SP,ML,G7W,200.0,2029-12-08,L,CANALIZACAO-SMP,4,"[(""Colunas"", ""Designação_Emissão""]), ..."


In [10]:
df_right = read_df('aero')
df_right

,Frequência,Latitude,Longitude,Entidade,Fonte,Multiplicidade
0,0.1143,-32.340055556,-54.223888889,NDB - MELO,AISGEO,1
1,0.2,-22.923333333,-42.0715,NDB - CABO FRIO,AISGEO,1
2,0.205,-5.386166667,-35.531,NDB - MAXARANGUAPE,AISGEO,1
3,0.21,-19.561,-46.964666667,NDB - ARAXA,AISGEO,1
4,0.225,-25.40866667,-54.62166667,NDB - ITAIPU,AISGEO,1
...,...,...,...,...,...,...
2990,995.0,-23.631379722,-46.652060278,DME - AEROPORTO DE CONGONHAS 34X,AISGEO,1
2991,995.0,-5.771808333,-35.350191667,DME - SAO GONCALO 34X,AISGEO,1
2992,995.0,-9.336946111,-54.952427778,DME - CACHIMBO 34X,AISGEO,1
2993,995.0,-9.509804722,-35.786944417,DME - IMC 34X,AISGEO,1


In [11]:
df_left = pd.concat([radcom, stel, srd, telecom, smp], ignore_index=True)
df_left

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
0,87.5,ASSOCIACAO COMUNITARIA VOZ DA LIBERDADE DE TUR...,50415095220,231,1008210959,-2.228611111111,-45.306666666666665,2112456,Turilândia,MA,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,87.5,ASSOCIACAO COMUNITARIA DO MORAD.DE ALVORADA DE...,50409064718,231,699491851,-18.734166666666667,-43.36472222222217,3102407,Alvorada de Minas,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,87.5,ASSOCIAÇÃO COMUNITARIA DA JUVENTUDE DE CONGONH...,50405625782,231,699359830,-18.812777777777665,-43.673611111111,3118106,Congonhas do Norte,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,87.5,ASSOCIACAO COMUNITARIA FOLHETA,50404381251,231,690859562,-18.95,-43.266666666666666,3122603,Dom Joaquim,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,87.5,ASSOCIAÇÃO DE RÁDIO COMUNITÁRIA DE CASTANHEIRA...,50411566547,231,1008401606,-11.137222222222167,-58.61333333333334,5102850,Castanheira,MT,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108514,1970.0,TELEFONICA BRASIL S.A.,50409146447,010,535800142,-23.1761,-48.1195,3540507,Porangaba,SP,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
108515,1767.5,TELEFONICA BRASIL S.A.,50409146447,010,535800142,-23.1761,-48.1195,3540507,Porangaba,SP,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
108516,829.5,TELEFONICA BRASIL S.A.,50409146447,010,535800142,-23.1761,-48.1195,3540507,Porangaba,SP,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
108517,845.75,TELEFONICA BRASIL S.A.,50409146447,010,689068093,-20.39,-49.4309,3535002,PALESTINA,SP,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [29]:
estacoes = read_df('estacoes')
estacoes

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
0,0.028,FURNAS CENTRAIS ELETRICAS S A,1030052263,19,1557670,-22.662779,-43.476391,3303500,Nova Iguaçu,RJ,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
1,0.030,FURNAS CENTRAIS ELETRICAS S A,1030052263,19,1558412,-23.709999,-46.273335,3530607,Mogi das Cruzes,SP,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
2,0.030,FURNAS CENTRAIS ELETRICAS S A,1030052263,19,1557823,-23.441668,-46.590832,3550308,São Paulo,SP,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
3,0.030,FURNAS CENTRAIS ELETRICAS S A,1030052263,19,859761,-22.926666,-43.264999,3304557,Rio de Janeiro,RJ,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
4,0.030,FURNAS CENTRAIS ELETRICAS S A,1030052263,19,859753,-22.774166,-47.004444,3509502,Campinas,SP,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542592,76000.000,ROBERT BOSCH LTDA,50440240239,35,1012776252,-22.898159,-47.112930,3509502,Campinas,SP,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
542593,76000.000,ROBERT BOSCH LTDA,50440240239,35,1012776210,-22.898159,-47.112930,3509502,Campinas,SP,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
542594,76000.000,ROBERT BOSCH LTDA,50440240239,35,1012776260,-22.898159,-47.112930,3509502,Campinas,SP,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
542595,76000.000,ROBERT BOSCH LTDA,50440240239,35,1012776236,-22.898159,-47.112930,3509502,Campinas,SP,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1


In [30]:
estacoes.columns[10:]

Index(['Classe', 'Classe_Emissão', 'Largura_Emissão(kHz)', 'Validade_RF',
       'Status', 'Fonte', 'Multiplicidade', 'Log', 'Cota_Base_Torre(m)',
       'Potência_Transmissor(W)', 'Ganho_Antena(dBd)',
       'Ângulo_Elevação_Antena', 'Azimute_Antena', 'Altura_Antena(m)',
       'Atenuação_Linha(db/100m)', 'Perdas_Acessórias_Linha(db)',
       'Padrão_Antena(dBd)', 'Comprimento_Linha(m)', 'Relatório_Canal'],
      dtype='object')

In [31]:
estacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542597 entries, 0 to 542596
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype                                                       
---  ------                       --------------   -----                                                       
 0   Frequência                   542597 non-null  double[pyarrow]                                             
 1   Entidade                     542597 non-null  string[pyarrow]                                             
 2   Fistel                       542597 non-null  uint64[pyarrow]                                             
 3   Serviço                      542597 non-null  uint16[pyarrow]                                             
 4   Estação                      542597 non-null  string[pyarrow]                                             
 5   Latitude                     542597 non-null  float[pyarrow]                                        

In [32]:
estacoes.Fonte.value_counts()

Fonte
MOSAICO-LICENCIAMENTO                    420047
CANALIZACAO-SMP                           80203
STEL                                      19595
MOSAICO-SRD                               14855
SRD                                        4964
STEL | ICAO                                 904
ICAO                                        694
CANALIZACAO-VOR/ILS/DME                     290
AISGEO                                      219
MOSAICO-LICENCIAMENTO | ICAO | AISWEB       168
RADAR                                       152
MOSAICO-LICENCIAMENTO | ICAO                148
STEL | ICAO | AISWEB                        142
AISWEB                                       64
ICAO | AISGEO                                44
ICAO | AISWEB                                36
STEL | AISWEB                                35
REDEMET                                      26
MOSAICO-LICENCIAMENTO | AISWEB               11
Name: count, dtype: int64[pyarrow]

In [33]:
for c in [
	'ICAO',
	'CANALIZACAO-VOR/ILS/DME',
	'AISGEO',
	'RADAR',
	'ICAO | AISWEB',
	'AISWEB',
	'ICAO | AISGEO',
	'REDEMET',
]:
	for col in [
		'Fistel',
		'Serviço',
		'Estação',
		'Classe',
		'Classe_Emissão',
		'Largura_Emissão(kHz)',
		'Validade_RF',
		'Status',
		'Fonte',
		'Multiplicidade',
		'Cota_Base_Torre(m)',
		'Potência_Transmissor(W)',
		'Ganho_Antena(dBd)',
		'Ângulo_Elevação_Antena',
		'Azimute_Antena',
		'Altura_Antena(m)',
		'Atenuação_Linha(db/100m)',
		'Perdas_Acessórias_Linha(db)',
		'Padrão_Antena(dBd)',
		'Comprimento_Linha(m)',
		'Relatório_Canal',
	]:
		assert estacoes.loc[estacoes.Fonte == c, col].nunique() == 1


In [39]:
for c in [
	'STEL | ICAO',
	'MOSAICO-LICENCIAMENTO | ICAO | AISWEB',
	'MOSAICO-LICENCIAMENTO | ICAO',
	'STEL | ICAO | AISWEB',
	'STEL | AISWEB',
	'MOSAICO-LICENCIAMENTO | AISWEB',
]:
	print(estacoes.loc[estacoes.Fonte == c, ['Fistel', 'Serviço', 'Estação']].sample(5))


Fistel  Serviço     Estação
18698  50418896356      507  1009939359
20870  50415723132      507  1007541307
19437  50415723132      507  1007666240
19510  50415723132      507  1007661485
18866  50415723132      507  1007724363

Fistel  Serviço     Estação
18905  11030016470       19   699524920
18623  11030016470       19  1001380638
19789  50413636844       19  1003159521
18479  11030016470       19   699526396
18396  11020237015       19     1068946

Fistel  Serviço    Estação
18602  11030016470       19  699524970
18499  50406881138       19  692616217
20131   1030091161       19  325127255
19888   2030398152       19  627454569
19901   2030398152       19  627453740

Fistel  Serviço     Estação
19475  50415723132      507  1007559222
18564  50415723132      507  1007685040
19461  50415723132      507  1007543024
18381  50415723132      507  1007696092
20113  50417804040      507   700065520

Fistel  Serviço     Estação
18913  50418861803      507  1009889874
19005  50415723132      507  1007533754
20702  50415723132      507  1007541412
19490  50415723132      507  1007558129
18966  50434034703      507  1011878086

Fistel  Serviço     Estação
19283  11030016470       19   699506905
19512  50415178436       19  1005507314
18550  11030016470       19   699525241
18832  11030016470       19   699526159
18831  11030016470       19   699506786

In [ ]:
df

,Frequência,Entidade_x,Fistel,Serviço,Estação,Latitude_x,Longitude_x,Código_Município,Município,UF,...,Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal,Latitude_y,Longitude_y,Entidade_y,Fonte_y,Multiplicidade_y,_merge
0,87.5,ASSOCIACAO COMUNITARIA VOZ DA LIBERDADE DE TUR...,50415095220,231,1008210959,-2.228611111111,-45.306666666666665,2112456,Turilândia,MA,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
1,87.5,ASSOCIACAO COMUNITARIA DO MORAD.DE ALVORADA DE...,50409064718,231,699491851,-18.734166666666667,-43.36472222222217,3102407,Alvorada de Minas,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
2,87.5,ASSOCIAÇÃO COMUNITARIA DA JUVENTUDE DE CONGONH...,50405625782,231,699359830,-18.812777777777665,-43.673611111111,3118106,Congonhas do Norte,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
3,87.5,ASSOCIACAO COMUNITARIA FOLHETA,50404381251,231,690859562,-18.95,-43.266666666666666,3122603,Dom Joaquim,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
4,87.5,ASSOCIAÇÃO DE RÁDIO COMUNITÁRIA DE CASTANHEIRA...,50411566547,231,1008401606,-11.137222222222167,-58.61333333333334,5102850,Castanheira,MT,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066219,995.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,-23.631379722,-46.652060278,DME - AEROPORTO DE CONGONHAS 34X,AISGEO,1,right_only
1066220,995.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,-5.771808333,-35.350191667,DME - SAO GONCALO 34X,AISGEO,1,right_only
1066221,995.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,-9.336946111,-54.952427778,DME - CACHIMBO 34X,AISGEO,1,right_only
1066222,995.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,-9.509804722,-35.786944417,DME - IMC 34X,AISGEO,1,right_only


In [ ]:
left_suffix, right_suffix = '_x', '_y'
lat, long = 'Latitude', 'Longitude'

left_only = df._merge == 'left_only'
right_only = df._merge == 'right_only'
both = df._merge == 'both'
df = df.drop(columns=['_merge'])

In [ ]:
only_left = df[left_only].copy()
only_left = only_left.iloc[:, :len(df_left.columns)]
only_left.columns = df_left.columns
only_left


,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
0,87.5,ASSOCIACAO COMUNITARIA VOZ DA LIBERDADE DE TUR...,50415095220,231,1008210959,-2.228611111111,-45.306666666666665,2112456,Turilândia,MA,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,87.5,ASSOCIACAO COMUNITARIA DO MORAD.DE ALVORADA DE...,50409064718,231,699491851,-18.734166666666667,-43.36472222222217,3102407,Alvorada de Minas,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,87.5,ASSOCIAÇÃO COMUNITARIA DA JUVENTUDE DE CONGONH...,50405625782,231,699359830,-18.812777777777665,-43.673611111111,3118106,Congonhas do Norte,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,87.5,ASSOCIACAO COMUNITARIA FOLHETA,50404381251,231,690859562,-18.95,-43.266666666666666,3122603,Dom Joaquim,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,87.5,ASSOCIAÇÃO DE RÁDIO COMUNITÁRIA DE CASTANHEIRA...,50411566547,231,1008401606,-11.137222222222167,-58.61333333333334,5102850,Castanheira,MT,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065599,1765.0,TELEFONICA BRASIL S.A.,50409146285,010,1011578503,-6.31095,-38.2823,2406007,<NA>,RN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1065600,1765.0,TELEFONICA BRASIL S.A.,50409146285,010,1011578511,-6.40588,-38.3899,2407005,<NA>,RN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1065601,1765.0,TELEFONICA BRASIL S.A.,50409146285,010,1011578520,-6.45823,-38.182,2414100,<NA>,RN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1065602,1765.0,TELEFONICA BRASIL S.A.,50409146285,010,1011578538,-5.71287,-38.1578,2311231,<NA>,CE,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
only_right = df[right_only].copy()
cols = ['Frequência'] + df.columns[len(df_left.columns):].to_list()
only_right = only_right.loc[:, cols]
only_right.columns = df_right.columns
only_right

,Frequência,Latitude,Longitude,Entidade,Fonte,Multiplicidade
1065604,0.1143,-32.340055556,-54.223888889,NDB - MELO,AISGEO,1
1065605,0.285,-21.267052778,-43.764111111,NDB - BARBACENA NDB NOT AVBL BEYOND 60 NM,AISGEO,1
1065606,0.285,-22.197333333,-54.928333333,NDB - DOURA NDB NOT AVBL BEYOND 70 NM,AISGEO,1
1065607,0.375,-19.016219444,-57.664455556,NDB - CORUMBA,AISGEO,1
1065608,0.385,-16.441666667,-39.0825,NDB - PORTO SEGURO NDB NOT AVBL BEYOND 27 NM,AISGEO,1
...,...,...,...,...,...,...
1066219,995.0,-23.631379722,-46.652060278,DME - AEROPORTO DE CONGONHAS 34X,AISGEO,1
1066220,995.0,-5.771808333,-35.350191667,DME - SAO GONCALO 34X,AISGEO,1
1066221,995.0,-9.336946111,-54.952427778,DME - CACHIMBO 34X,AISGEO,1
1066222,995.0,-9.509804722,-35.786944417,DME - IMC 34X,AISGEO,1
